In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(os.path.join("data", "db", "CleanedObservations.csv"))
df.head()

,Player Id,Year,Position,Adjusted Salary,Log Adjusted Salary,Adjusted Team Payroll,Batting_Career_Num_Seasons,Batting_Career_G,Batting_Career_SB,Batting_Career_RBI,...,Num_Post_Season_Appearances,Num_All_Star_Appearances,0.0,1B,2B,3B,C,MULTIPLE,P,SS
0,blanche01,2011,C,0.296,-1.218,0.221,0.522,0.297,0.008,0.158,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,bloomwi01,2011,MULTIPLE,0.266,-1.323,0.221,0.348,0.257,0.176,0.086,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,blumge01,2011,MULTIPLE,0.399,-0.918,0.221,0.478,0.464,0.032,0.280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,branyru01,2011,1B,0.296,-1.218,0.221,0.522,0.341,0.024,0.271,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,demelsa01,2011,P,0.123,-2.092,0.221,0.000,0.012,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


##  Regression on "Log Adjusted Salary"

We test LinearRegression on the log(y), where y is the Adjusted Salary:

In [7]:
players = df['Player Id'].unique()

from sklearn import linear_model
from sklearn import cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

players = df['Player Id'].unique()

columns = ['Adjusted Team Payroll', 
           'Batting_Career_Num_Seasons', 
           'Batting_Career_HR',
           'Batting_Career_SB',
           'Batting_Career_RBI',
           'Pitching_Career_SO',
           'Pitching_Career_ERA',
           'Fielding_Career_A',
           'Fielding_Career_PO',
           'Fielding_Career_E',
           'Fielding_Career_G',
           '0.0', '1B', '2B', '3B', 'C', 'MULTIPLE', 'P', 'SS']
regr = linear_model.LinearRegression()

X = np.asarray(pd.DataFrame(df, columns=columns))
y = np.asarray(df['Log Adjusted Salary'])

from sklearn.linear_model import LinearRegression
regr = LinearRegression()

# Simple train/test split
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.3, random_state=55)
regr.fit(x_train, y_train)
score = regr.score(x_test, y_test)
print(score)

# Get groups for GroupKFold
df.sort(['Player Id'], inplace=True)
players = list(df['Player Id'].values)
groups = [players.index(row['Player Id']) for index, row in df.iterrows()]

# 5-fold CV:
scores = cross_val_score(regr, X, y, groups, cv=GroupKFold(n_splits=5))
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

0.601713736421


C:\Users\Owner\Anaconda3\envs\data_analysis\lib\site-packages\ipykernel\__main__.py:37: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Accuracy: 0.56 (+/- 0.19)


## Validating the accuracy of our log-linear model

Implement predict(X) for a LogLinearRegression class that extends Linear Regression

Adjust the predictions as described here: http://davegiles.blogspot.com/2013/08/forecasting-from-log-linear-regressions.html

Compare the results of LogLinearRegression.predict(X) and exp(LinearRegression.predict(X)).

In [6]:
from sklearn.linear_model import LinearRegression
class LogLinearRegression(LinearRegression):
    def fit(self, x, y):
        self.actuals = y
        LinearRegression.fit(self, x, y)
        
    def predict(self, X):
        y = LinearRegression.predict(self, X)
        sum_of_sq_err = 0
        for i, prediction in enumerate(y):
           sum_of_sq_err = np.square(prediction - self.actuals[i])
        
        return np.exp(y + (sum_of_sq_err / (len(y) - len(self.coef_))) / 2)
    

regr = LogLinearRegression()
regr.fit(x_train, y_train)
predictions1 = regr.predict(x_train)
print(predictions1)

regr = LinearRegression()
regr.fit(x_train, y_train)
predictions2 = np.exp(regr.predict(x_train))
print(predictions2)

print(predictions1 - predictions2)


[ 0.47935687  0.24483153  0.60596386 ...,  0.27366329  0.60183566
  0.1849891 ]
[ 0.47933691  0.24482134  0.60593863 ...,  0.27365189  0.6018106   0.1849814 ]
[  1.99557280e-05   1.01923885e-05   2.52264038e-05 ...,   1.13926606e-05
   2.50545458e-05   7.70113546e-06]


The differences between the predicted values are close enough. We can just rely on the prediction y from LinearRegression for our log(y) model, and to get the predicted value just take exp(y).